In [1]:
import tensorflow as tf

In [2]:
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("./data/mnist", one_hot=True)

Extracting ./data/mnist/train-images-idx3-ubyte.gz
Extracting ./data/mnist/train-labels-idx1-ubyte.gz
Extracting ./data/mnist/t10k-images-idx3-ubyte.gz
Extracting ./data/mnist/t10k-labels-idx1-ubyte.gz


### 신경망 모델을 만들자

In [4]:
x_true = tf.placeholder(tf.float32, [None, 28, 28, 1])
y_true = tf.placeholder(tf.float32, [None, 10])

Hidden layer #1

In [6]:
weight_1 = tf.Variable(tf.truncated_normal([3,3,1,32], stddev=0.01))
hidden_1 = tf.nn.conv2d(x_true, weight_1, strides=[1,1,1,1], padding='SAME')
hidden_1 = tf.nn.relu(hidden_1)
hidden_1 = tf.nn.max_pool(hidden_1, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')
hidden_1 = tf.nn.dropout(hidden_1, keep_prob=0.8)

Hidden layer #2

In [7]:
weight_2 = tf.Variable(tf.truncated_normal([3,3,32,64], stddev=0.01))
hidden_2 = tf.nn.conv2d(hidden_1, weight_2, strides=[1,1,1,1], padding='SAME')
hidden_2 = tf.nn.relu(hidden_2)
hidden_2 = tf.nn.max_pool(hidden_2, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

FC 레이어로 가기 위해 hidden_2의 차원을 줄여줍니다.

In [9]:
hidden_2 = tf.reshape(hidden_2, [-1,7*7*64]) # -1은 나머지 전체를 의미
hidden_2 = tf.nn.dropout(hidden_2, keep_prob=0.8)

FC layer

In [11]:
weight_3 = tf.Variable(tf.truncated_normal([7*7*64, 256], stddev=0.01))
fc_1 = tf.matmul(hidden_2, weight_3)
fc_1 = tf.nn.relu(fc_1)
fc_1 = tf.nn.dropout(fc_1, 0.5)

softmax layer

In [12]:
weight_4 = tf.Variable(tf.truncated_normal([256,10], stddev=0.01))
y_pred = tf.matmul(fc_1, weight_4)

In [13]:
cost = tf.nn.softmax_cross_entropy_with_logits(logits=y_pred, labels=y_true)
cost = tf.reduce_mean(cost)
optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
optimizer = optimizer.minimize(cost)

### 모델 학습시키기

In [14]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [15]:
batch_size = 100
num_batch = int(mnist.train.num_examples/batch_size)

지금까지 했던 일반 NN과는 비교가 안 되게 오랜 시간이 걸림에 유의합니다.

In [18]:
for epoch in range(10):
    total_cost = 0
    for i in range(num_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        # CNN 모델을 위한 자료형인 [28,28,1]의 형태로 reshaping
        batch_xs = batch_xs.reshape(-1,28,28,1)
        _, cost_value = sess.run([optimizer, cost], 
                           {x_true:batch_xs, y_true:batch_ys})
        total_cost += cost_value
    print "Epoch : {0}, Cost : {1}".format(epoch + 1, total_cost/num_batch)
print "Optimization completed."

Epoch : 1, Cost : 0.29902660844
Epoch : 2, Cost : 0.153112152016
Epoch : 3, Cost : 0.145488638011
Epoch : 4, Cost : 0.146243524152
Epoch : 5, Cost : 0.139610956757
Epoch : 6, Cost : 0.144187849213
Epoch : 7, Cost : 0.142065617987
Epoch : 8, Cost : 0.139127645681
Epoch : 9, Cost : 0.132123957083
Epoch : 10, Cost : 0.130543615264
Optimization completed.


In [19]:
correct_prediction = tf.equal(tf.argmax(y_pred,1), tf.argmax(y_true,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print "정확도 : {0}".format(sess.run(accuracy, 
                                  {x_true:mnist.test.images.reshape(-1, 28, 28, 1), 
                                   y_true:mnist.test.labels}))

정확도 : 0.963900029659
